# Assignment 4: Simple Sentiment Analysis with PyTorch

**Goal:** Your task is to complete the code in this notebook to build and train a basic neural network for binary sentiment classification (positive/negative) using PyTorch.

**Instructions:**

1.  Read through the explanations in each section carefully.
2.  Find the code blocks marked with `# TODO:` or `YOUR CODE HERE`.
3.  Fill in the missing code according to the instructions.
4.  Run the notebook sequentially. **Do not change the `torch.manual_seed(42)` line**, as it's crucial for auto-scoring.
5.  After completing a section, run the corresponding "✅ Check Your Work" cell to see if your implementation is correct.
6. If you don't get full credit for a block, go back, edit your code in that block, and rerun it (along with any dependent cells) until you pass the check.
7.  Your final score will be based on passing these checks.

**Let's get started!**

## 1. Setup

First, let's import the required PyTorch libraries and other utilities. This part is already done for you.
"""

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import os, re, random
import numpy as np
import torch.nn.functional as F

# --- DO NOT CHANGE THIS SEED ---
# Set random seed for reproducibility and scoring
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
# -------------------------------

## 2. Data

We'll use a very small, hardcoded dataset for simplicity.

*   `1` represents positive sentiment.
*   `0` represents negative sentiment.

In [2]:
raw_data = [
    ("really good movie", 1), ("absolutely loved it", 1), ("such a great film", 1), ("wonderful and touching", 1),
    ("incredible story", 1), ("very enjoyable experience", 1), ("loved every moment", 1), ("heartwarming and sweet", 1),
    ("amazing acting", 1), ("fantastic film overall", 1), ("excellent direction", 1), ("great plot", 1),
    ("a masterpiece", 1), ("beautifully made", 1), ("just perfect", 1), ("top notch acting", 1),
    ("brilliant and emotional", 1), ("superb movie experience", 1), ("highly recommend this", 1), ("best film I've seen", 1),
    ("great visuals", 1), ("emotional and inspiring", 1), ("outstanding in every way", 1), ("so much fun", 1),
    ("pleasantly surprised", 1), ("joyful and uplifting", 1), ("flawless performance", 1), ("great positive message", 1),
    ("positive feelings only", 1), ("a must watch movie", 1), ("truly captivating story", 1), ("loved the music", 1),
    ("looked stunning", 1), ("genuinely funny parts", 1), ("a feel good movie", 1), ("kept me engaged", 1),
    ("great chemistry between actors", 1), ("clever writing", 1), ("visually impressive", 1), ("exceeded expectations", 1),
    ("a very satisfying watch", 1), ("powerful performances", 1), ("unique and original", 1), ("well developed story", 1),
    ("the ending was perfect", 1), ("charming and delightful", 1), ("a real gem of a film", 1), ("thought provoking and deep", 1),
    ("laugh out loud funny", 1), ("skillfully directed", 1), ("excellent pacing", 1), ("incredibly well acted", 1),
    ("beautiful musical score", 1), ("a heartwarming story", 1), ("a visual treat", 1), ("genuinely moving", 1),
    ("great special effects", 1), ("loved the dialogue", 1), ("memorable characters", 1), ("entertaining from start to finish", 1),
    ("a fantastic journey", 1), ("will watch it again", 1), ("highly imaginative plot", 1), ("superbly crafted", 1),
    ("full of charm", 1), ("refreshingly different", 1), ("strong emotional connection", 1), ("genuinely touching moments", 1),
    ("brilliant execution", 1), ("a delightful surprise", 1), ("outstanding writing", 1), ("perfect casting", 1),
    ("loved the overall atmosphere", 1), ("truly inspiring", 1), ("masterful storytelling", 1), ("an absolute joy to watch", 1),
    ("well worth the time", 1), ("great character development", 1),

    ("really bad movie", 0), ("absolutely hated it", 0), ("such a boring film", 0), ("terrible and slow", 0),
    ("weak story", 0), ("very disappointing experience", 0), ("hated every moment", 0), ("cringe worthy and dull", 0),
    ("awful acting", 0), ("horrible film overall", 0), ("poor direction", 0), ("bad plot", 0),
    ("a total disaster", 0), ("badly made movie", 0), ("just plain awful", 0), ("low quality acting", 0),
    ("stupid and annoying", 0), ("painful movie experience", 0), ("do not recommend this", 0), ("worst film I've seen", 0),
    ("ugly visuals", 0), ("dry and boring", 0), ("complete waste of time", 0), ("so much cringe", 0),
    ("deeply disappointed", 0), ("frustrating and flat", 0), ("terrible negative message", 0), ("negative feelings only", 0),
    ("a must skip movie", 0), ("not worth watching", 0), ("truly dreadful story", 0), ("hated the music", 0),
    ("looked jarring", 0), ("genuinely unfunny parts", 0), ("a depressing movie", 0), ("lost my interest", 0),
    ("zero chemistry between actors", 0), ("lazy writing", 0), ("visually unappealing", 0), ("failed expectations", 0),
    ("a very unsatisfying watch", 0), ("weak performances", 0), ("unoriginal and derivative", 0), ("poorly developed story", 0),
    ("the ending was terrible", 0), ("awkward and unpleasant", 0), ("a real dud of a film", 0), ("nonsensical and confusing", 0),
    ("trying too hard", 0), ("clumsily directed", 0), ("terrible pacing", 0), ("incredibly poorly acted", 0),
    ("annoying musical score", 0), ("a tedious story", 0), ("an ugly mess", 0), ("genuinely irritating", 0),
    ("bad special effects", 0), ("hated the dialogue", 0), ("forgettable characters", 0), ("boring from start to finish", 0),
    ("a painful journey", 0), ("will avoid watching again", 0), ("completely unimaginative plot", 0), ("poorly crafted", 0),
    ("lacked any charm", 0), ("predictable and stale", 0), ("no emotional depth", 0), ("forced and fake moments", 0),
    ("terrible execution", 0), ("an unpleasant surprise", 0), ("awful writing", 0), ("terrible casting", 0),
    ("hated the overall atmosphere", 0), ("truly pointless", 0), ("confusing storytelling", 0), ("an absolute pain to watch", 0),
    ("not worth the money", 0), ("weak character development", 0)
]

# Separate texts and labels - Provided
texts = [text for text, label in raw_data]
labels = [label for text, label in raw_data]

print("Sample Texts:", texts[:3])
print("Sample Labels:", labels[:3])

Sample Texts: ['really good movie', 'absolutely loved it', 'such a great film']
Sample Labels: [1, 1, 1]


## 3. Preprocessing

Text data needs to be converted into numbers that our neural network can understand. This section is mostly provided.

### 3.1. Tokenization and Cleaning

In [4]:
# Provided
def simple_tokenizer(text):
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text) # Keep only letters and spaces
    tokens = text.split()
    return tokens

# Tokenize all texts - Provided
tokenized_texts = [simple_tokenizer(text) for text in texts]
print("Tokenized Sample:", tokenized_texts[0])

Tokenized Sample: ['really', 'good', 'movie']


### 3.2. Build Vocabulary

Create a mapping from unique words to integer indices. We'll add special tokens: `<PAD>` for padding (though `EmbeddingBag` handles variable lengths nicely) and `<UNK>` for unknown words encountered later.

In [5]:
# Provided
# Count word frequencies
word_counts = Counter(token for text in tokenized_texts for token in text)

# Create vocabulary (mapping word to index)
vocab = {"<PAD>": 0, "<UNK>": 1}
vocab.update({word: i+2 for i, (word, count) in enumerate(word_counts.items())})

vocab_size = len(vocab)
print("Vocabulary Size:", vocab_size)
print("Sample Vocab:", list(vocab.items())[:10])

Vocabulary Size: 234
Sample Vocab: [('<PAD>', 0), ('<UNK>', 1), ('really', 2), ('good', 3), ('movie', 4), ('absolutely', 5), ('loved', 6), ('it', 7), ('such', 8), ('a', 9)]


### 3.3. Numericalize Text

Convert each tokenized sentence into a sequence of integers using the vocabulary.

In [ ]:
# Provided
def numericalize(tokens, vocab):
    return [vocab.get(token, 1) for token in tokens] # Use <UNK> index (1) if word is not in vocab

numericalized_texts = [numericalize(tokens, vocab) for tokens in tokenized_texts]
print("Numericalized Sample:", numericalized_texts[0])

### 3.4. Encode Labels

Our labels (0 and 1) are already numerical, but we'll convert them to tensors.

In [ ]:
# Provided
# Convert labels to tensors
encoded_labels = torch.tensor(labels, dtype=torch.float32) # Use float for BCEWithLogitsLoss
print("Encoded Labels Sample:", encoded_labels[:3])

## 4. Dataset & DataLoader

We need a way to efficiently load and batch our data during training. This is provided for you.

*   **`SentimentDataset`**: A custom class that holds our numericalized texts and labels.
*   **`collate_fn`**: A function used by the `DataLoader` to process a batch of data points. It handles sequences of varying lengths and prepares them for the `EmbeddingBag` layer by creating offsets.
*   **`DataLoader`**: Manages batching and shuffling.

In [ ]:
# Provided
class SentimentDataset(Dataset):
    def __init__(self, numericalized_texts, labels):
        self.texts = numericalized_texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Return text as a tensor
        return torch.tensor(self.texts[idx], dtype=torch.int64), self.labels[idx]

# Provided
# Collate function to handle variable length sequences for EmbeddingBag
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_text, _label) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(_text, dtype=torch.int64)
        text_list.append(processed_text)
        # Track the starting index of each sequence in the concatenated tensor
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.float32)
    # Concatenate all sequences into a single tensor
    text_list = torch.cat(text_list)

    # Convert offsets to a tensor; remove the last element as it's the total length
    offsets = torch.tensor(offsets[:-1], dtype=torch.int64)
    return text_list, label_list, offsets

# Create the dataset instance
dataset = SentimentDataset(numericalized_texts, encoded_labels)

# Create the DataLoader
batch_size = 4 # Small batch size for our small dataset
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_batch,
    worker_init_fn=np.random.seed(SEED) # Ensure dataloader shuffle is deterministic
)

# Example of one batch (optional check)
try:
  text_batch, label_batch, offset_batch = next(iter(dataloader))
  print("--- Sample Batch --- ")
  print("Text Tensor: ", text_batch)
  print("Labels Tensor:", label_batch)
  print("Offsets Tensor:", offset_batch)
  print("--------------------")
except StopIteration:
  print("Dataloader is empty or batch size is larger than dataset size.")

--- Sample Batch --- 
Text Tensor:  tensor([  9,  17,  89,  66,  49,  84,  39,  13,  48, 206,  95,  16])
Labels Tensor: tensor([1., 1., 1., 0.])
Offsets Tensor: tensor([0, 4, 2, 3])
--------------------


<ipython-input-34-f9a31910404c>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


## 5. Model Definition (70 Points)

**Task 1:** Define the neural network architecture (**40 points**).

*   You need to initialize the layers in the `__init__` method:
    *   An `nn.EmbeddingBag` layer. Use the provided `vocab_size` and `embed_dim`. Set `sparse=False` and `mode="mean"`.
    *   A `nn.Linear` layer for the hidden layer. It should map from `embed_dim` to `hidden_dim`.
    *   An `nn.Dropout` layer. Use the provided `dropout` rate.
    *   A final `nn.Linear` layer for the output. It should map from `hidden_dim` to `num_class`.
*   You need to implement the `forward` method (**30 points**):
    *   Pass the `text` and `offsets` through the `embedding` layer.
    *   Pass the result through the `hidden` layer, followed by a `ReLU` activation function (`F.relu`).
    *   Apply `dropout` to the hidden layer's output.
    *   Pass the result through the `output` layer.
    *   Return the final output (these will be logits).

In [ ]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class, dropout=0.5):
        super(SentimentNet, self).__init__()
        self.embedding = None  # TODO: Initialize the nn.EmbeddingBag layer (10 Point)
        self.hidden = None    # TODO: Initialize the first nn.Linear hidden layer (10 Point)
        self.dropout = None   # TODO: Initialize the nn.Dropout layer (10 Point)
        self.output = None    # TODO: Initialize the final nn.Linear output layer (10 Point)

        # --- YOUR CODE HERE (~4 Lines) ---
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False, mode="mean") # YOUR CODE HERE
        self.hidden = nn.Linear(embed_dim, hidden_dim) # YOUR CODE HERE
        self.dropout = nn.Dropout(p=dropout) # YOUR CODE HERE
        self.output = nn.Linear(hidden_dim, num_class) # YOUR CODE HERE
        # --- END YOUR CODE ---


    def forward(self, text, offsets):
        embedded = None # Placeholder
        hidden_out = None # Placeholder
        output = None # Placeholder

        # --- YOUR CODE HERE (Implement forward pass) (~4 Lines, 30 points) ---
        # 1. Apply embedding layer
        embedded = self.embedding(text, offsets) # YOUR CODE HERE

        # 2. Pass through hidden layer and ReLU activation
        hidden_out = F.relu(self.hidden(embedded)) # YOUR CODE HERE

        # 3. Apply dropout
        hidden_out = self.dropout(hidden_out) # YOUR CODE HERE

        # 4. Pass through the output layer
        output = self.output(hidden_out)  # YOUR CODE HERE
        # --- END YOUR CODE ---

        return output

# Model Hyperparameters (Provided)
EMBED_DIM = 256
HIDDEN_DIM = 256
NUM_CLASS = 1 # For binary classification
DROP_RATE = 0.2

# Instantiate the model (using your definition above)
model = SentimentNet(vocab_size, embed_dim=EMBED_DIM, hidden_dim=HIDDEN_DIM, num_class=NUM_CLASS, dropout=DROP_RATE)
print("Model Architecture:")

print(model)

Model Architecture:
SentimentNet(
  (embedding): EmbeddingBag(234, 256, mode='mean')
  (hidden): Linear(in_features=256, out_features=256, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Linear(in_features=256, out_features=1, bias=True)
)


### ✅ Check Your Work: Task 1 - Model Definition

In [ ]:
# Check Task 1 (Model Definition)
points_task1 = 0
target_output_shape = torch.Size([batch_size, NUM_CLASS]) # Expect (batch_size, 1)
try:
    # Get a sample batch
    torch.manual_seed(SEED) # Reset seed just before getting batch
    dataloader_iter = iter(DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch, worker_init_fn=np.random.seed(SEED)))
    text_batch, _, offset_batch = next(dataloader_iter)

    # Check if layers are initialized
    if isinstance(model.embedding, nn.EmbeddingBag) and \
       isinstance(model.hidden, nn.Linear) and \
       isinstance(model.dropout, nn.Dropout) and \
       isinstance(model.output, nn.Linear):
       points_task1 += 40 # 10 point per layer init
       print("[Task 1] Layer Initialization: Correct! (+40 points)")
    else:
       print("[Task 1] Layer Initialization: Incorrect. Check nn.EmbeddingBag, nn.Linear, nn.Dropout initializations.")

    # Check forward pass output shape
    model.eval() # Set to eval mode for checking
    with torch.no_grad():
        output = model(text_batch, offset_batch)
        if output.shape == target_output_shape:
             points_task1 += 30
             print(f"[Task 1] Forward Pass Output Shape: Correct! ({output.shape}) (+30 point)")
        else:
             print(f"[Task 1] Forward Pass Output Shape: Incorrect. Expected {target_output_shape}, got {output.shape}.")

except Exception as e:
    print(f"[Task 1] Error during check: {e}")
    print("Make sure your layer dimensions and forward pass logic are correct.")

[Task 1] Layer Initialization: Correct! (+40 points)
[Task 1] Forward Pass Output Shape: Correct! (torch.Size([4, 1])) (+30 point)


<ipython-input-34-f9a31910404c>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


## 6. Training Setup

**Task 2:** Set up the loss function and optimizer. **(20 Points Total)**

*   Instantiate the loss function: Use `nn.BCEWithLogitsLoss`, which is suitable for binary classification with logits output. (10 Point)
*   Instantiate the optimizer: Use `optim.Adam`, passing the `model.parameters()` and the specified `learning_rate`. (10 Point)

In [ ]:
# Training Hyperparameters (Provided)
learning_rate = 1e-4
num_epochs = 100

# --- YOUR CODE HERE (~2 Lines)---
criterion = nn.BCEWithLogitsLoss() # TODO: Define the loss function (Use BCEWithLogitsLoss) (10 Point)
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # TODO: Define the optimizer (Use Adam) (10 Point)
# --- END YOUR CODE ---

### ✅ Check Your Work: Task 2 - Loss & Optimizer

In [ ]:
# Check Task 2 (Loss and Optimizer)
points_task2 = 0
try:
    if isinstance(criterion, nn.BCEWithLogitsLoss):
        points_task2 += 10
        print("[Task 2] Criterion Definition: Correct! (+10 point)")
    else:
        print("[Task 2] Criterion Definition: Incorrect. Should be nn.BCEWithLogitsLoss.")

    if isinstance(optimizer, optim.Adam):
         if len(optimizer.param_groups) > 0 and len(optimizer.param_groups[0]['params']) > 0:
             points_task2 += 10
             print("[Task 2] Optimizer Definition: Correct! (+10 point)")
         else:
             print("[Task 2] Optimizer Definition: Potentially incorrect. Did you pass model.parameters()?")
    else:
         print("[Task 2] Optimizer Definition: Incorrect. Should be optim.Adam.")

except Exception as e:
    print(f"[Task 2] Error during check: {e}")

[Task 2] Criterion Definition: Correct! (+10 point)
[Task 2] Optimizer Definition: Correct! (+10 point)


## 7. Training Loop (Provided)

This section contains the complete training loop. Read through it to understand the process. No tasks here.

In [ ]:
print("\nStarting Training...")
# Ensure model, criterion, optimizer are defined from previous steps
if 'model' not in locals() or model is None or criterion is None or optimizer is None:
     print("ERROR: Model, criterion, or optimizer not defined. Please complete previous steps.")
else:
    # Make sure the optimizer is linked to the *current* model instance from Task 1 check reset
    # If Task 2 was correct, this re-links the same type of optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    torch.manual_seed(SEED) # Ensure consistent training start

    training_losses = []
    final_epoch_acc = 0

    for epoch in range(num_epochs):
        model.train() # Set model to training mode
        total_loss = 0
        total_correct = 0
        total_samples = 0

        # We rely on the DataLoader's shuffle seeded correctly at initialization
        for texts_batch, labels_batch, offsets_batch in dataloader:

            # Skip batch if texts_batch is empty
            if texts_batch.numel() == 0: continue

            # Zero gradients
            optimizer.zero_grad()

            # --- Training Steps ---
            # 1. Forward pass
            outputs = model(texts_batch, offsets_batch) # Shape: (batch_size, 1)

            # 2. Squeeze output for the loss function
            outputs_squeezed = outputs.squeeze(1) # Shape: (batch_size)

            # 3. Calculate loss
            loss = criterion(outputs_squeezed, labels_batch)

            # 4. Backward pass
            loss.backward()

            # 5. Optimize
            optimizer.step()

            # 6. Calculate accuracy
            predicted = (torch.sigmoid(outputs_squeezed) > 0.5).float()
            # --- End Training Steps ---

            # Track metrics
            total_loss += loss.item() * labels_batch.size(0)
            total_correct += (predicted == labels_batch).sum().item()
            total_samples += labels_batch.size(0)

        # Calculate average loss and accuracy for the epoch
        if total_samples > 0:
            epoch_loss = total_loss / total_samples
            epoch_acc = total_correct / total_samples
            training_losses.append(epoch_loss)
            final_epoch_acc = epoch_acc
        else:
            epoch_loss = 0
            epoch_acc = 0

        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    print("\nTraining finished.")


Starting Training...


<ipython-input-34-f9a31910404c>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


Epoch [1/100], Loss: 0.6956, Accuracy: 0.4615
Epoch [5/100], Loss: 0.6750, Accuracy: 0.5513
Epoch [10/100], Loss: 0.6737, Accuracy: 0.6090
Epoch [15/100], Loss: 0.6428, Accuracy: 0.6346
Epoch [20/100], Loss: 0.6357, Accuracy: 0.6667
Epoch [25/100], Loss: 0.6114, Accuracy: 0.6538
Epoch [30/100], Loss: 0.6492, Accuracy: 0.6026
Epoch [35/100], Loss: 0.6224, Accuracy: 0.6410
Epoch [40/100], Loss: 0.6079, Accuracy: 0.6667
Epoch [45/100], Loss: 0.5790, Accuracy: 0.6603
Epoch [50/100], Loss: 0.5939, Accuracy: 0.6218
Epoch [55/100], Loss: 0.5927, Accuracy: 0.6538
Epoch [60/100], Loss: 0.5542, Accuracy: 0.6987
Epoch [65/100], Loss: 0.5742, Accuracy: 0.7051
Epoch [70/100], Loss: 0.5543, Accuracy: 0.6538
Epoch [75/100], Loss: 0.5753, Accuracy: 0.6859
Epoch [80/100], Loss: 0.5741, Accuracy: 0.6603
Epoch [85/100], Loss: 0.5078, Accuracy: 0.6667
Epoch [90/100], Loss: 0.5241, Accuracy: 0.6859
Epoch [95/100], Loss: 0.5082, Accuracy: 0.6795
Epoch [100/100], Loss: 0.5061, Accuracy: 0.6859

Training fini

## 8. Evaluation (Provided)

Evaluate your trained model on unseen test data. No tasks here, just run the cell.

In [ ]:
def evaluate(model, dataloader, criterion):
    model.eval() # Set model to evaluation mode
    total_loss = 0
    total_correct = 0
    total_samples = 0

    with torch.no_grad(): # Disable gradient calculation
        for texts_batch, labels_batch, offsets_batch in dataloader:

            outputs = model(texts_batch, offsets_batch).squeeze(1)
            loss = criterion(outputs, labels_batch)

            total_loss += loss.item() * labels_batch.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_correct += (predicted == labels_batch).sum().item()
            total_samples += labels_batch.size(0)

    if total_samples > 0:
        avg_loss = total_loss / total_samples
        avg_acc = total_correct / total_samples
        print(f'Evaluation on training data: Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')
    else:
        print("Evaluation failed: No samples processed.")

# Define test examples
test_sentences = [
    "strong emotional core",
    "genuinely touching moments",
    "brilliant execution",
    "a delightful surprise",
    "outstanding screenplay",
    "perfect casting choice",
    "loved the atmosphere",
    "truly inspiring message",
    "masterful storytelling",
    "an absolute joy",
    "well worth the time",
    "great character arcs",
    "the music was perfect",
    "fantastic world building",

    "no emotional core",
    "forced and fake moments",
    "terrible execution",
    "an unpleasant surprise",
    "awful screenplay",
    "terrible casting choice",
    "hated the atmosphere",
    "truly pointless message",
    "confusing storytelling",
    "an absolute pain",
    "not worth the money",
    "weak character arcs",
    "the music was awful",
    "lazy world building",
]
test_labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Tokenize and numericalize
test_tokenized = [simple_tokenizer(sent) for sent in test_sentences]
test_numericalized = [numericalize(tokens, vocab) for tokens in test_tokenized]
test_encoded_labels = torch.tensor(test_labels, dtype=torch.float32)

# Create test dataset and dataloader
test_dataset = SentimentDataset(test_numericalized, test_encoded_labels)
test_loader = DataLoader(test_dataset, batch_size=4, collate_fn=collate_batch)

# Evaluate the trained model
evaluate(model, test_loader, criterion)

Evaluation on training data: Loss: 0.5160, Accuracy: 0.6071


<ipython-input-34-f9a31910404c>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_text = torch.tensor(_text, dtype=torch.int64)


## 9. Prediction Function

Let's create a function to predict the sentiment of a new, unseen sentence.

In [ ]:
def predict_sentiment(text, model, vocab, tokenizer_func):
    model.eval() # Set model to evaluation mod
    # 1. Tokenize and numericalize
    tokens = tokenizer_func(text)
    numericalized = numericalize(tokens, vocab)

    # 2. Convert to tensor
    text_tensor = torch.tensor(numericalized, dtype=torch.int64)

    # 3. Create offsets tensor for a single sequence (starts at 0)
    # This is crucial for EmbeddingBag when processing a single item
    offsets = torch.tensor([0], dtype=torch.int64)

    # 4. Make prediction
    with torch.no_grad():
        # The model expects batch dimension implicitly via offsets for EmbeddingBag
        output = model(text_tensor, offsets)
        # Apply sigmoid to get probability
        probability = torch.sigmoid(output).item()

    # 5. Interpret result
    sentiment = "Positive" if probability > 0.5 else "Negative"
    probability = probability if sentiment == "Positive" else 1 - probability
    return f"{sentiment} (Probability: {probability:.4f})"

# --- Test the prediction function ---
print("-" * 30)
test_sentence1 = "An excellent movie."
print(f"Sentence: '{test_sentence1}'")
print(f"Prediction: {predict_sentiment(test_sentence1, model, vocab, simple_tokenizer)}")
print("-" * 30)

test_sentence2 = "That was a total waste of time."
print(f"Sentence: '{test_sentence2}'")
print(f"Prediction: {predict_sentiment(test_sentence2, model, vocab, simple_tokenizer)}")
print("-" * 30)

test_sentence3 = "Mediocre plot but decent acting."
print(f"Sentence: '{test_sentence3}'")
print(f"Prediction: {predict_sentiment(test_sentence3, model, vocab, simple_tokenizer)}")
print("-" * 30)

test_sentence4 = "Absolutely loved every second!"
print(f"Sentence: '{test_sentence4}'")
print(f"Prediction: {predict_sentiment(test_sentence4, model, vocab, simple_tokenizer)}")
print("-" * 30)

test_sentence5 = "The acting was poor and the story made no sense."
print(f"Sentence: '{test_sentence5}'")
print(f"Prediction: {predict_sentiment(test_sentence5, model, vocab, simple_tokenizer)}")
print("-" * 30)

------------------------------
Sentence: 'An excellent movie.'
Prediction: Positive (Probability: 0.5052)
------------------------------
Sentence: 'That was a total waste of time.'
Prediction: Negative (Probability: 0.6919)
------------------------------
Sentence: 'Mediocre plot but decent acting.'
Prediction: Negative (Probability: 0.7711)
------------------------------
Sentence: 'Absolutely loved every second!'
Prediction: Positive (Probability: 0.6992)
------------------------------
Sentence: 'The acting was poor and the story made no sense.'
Prediction: Negative (Probability: 0.6228)
------------------------------


## 🏁 Total Score

In [ ]:
assignment_score = points_task1 + points_task2
TOTAL_POINTS = 100
print("\n" + "="*30)
print(f"🏁 Your final score: {assignment_score:.1f} / {TOTAL_POINTS}")


🏁 Your final score: 90.0 / 100


## Task 3: Experiment with Learning Rate (10 Points)
You’ve reached the final question! To receive full credit (**10 points remaining**), you need to experiment with the learning rate. Please go back to the model definition block and rerun it (to re-initialize your model). Then, run all subsequent code blocks up to the learning rate definition block. **Change the learning rate to `1e-3`**, and rerun the optimizer, training, and evaluation blocks using this new learning rate.  

After training, report the following results below: **Train Loss**, **Train Accuracy**, **Test Loss**, **Test Accuracy**  

**Instructions:**
1. Re-initialize your model by rerunning the model definition block.
2. In the learning rate definition block, set `learning_rate = 1e-3`.
3. Rerun the optimizer, loss function, training loop, and evaluation code.
4. Fill in your results above.

---

**Tip:**  
If you make a mistake or want to try again, you can always rerun the relevant blocks as many times as needed to improve your results!

In [ ]:
train_loss = 0.5136  # YOUR CODE HERE
train_accuracy = 0.6859  # YOUR CODE HERE
test_loss = 0.5019  # YOUR CODE HERE
test_accuracy = 0.6071  # YOUR CODE HERE